In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Initialize

openai = OpenAI()
MODEL = "gpt-4o-mini"

In [4]:
system_message = "You are a helpful assistant"

## Reminder of the structure of prompt messages to OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

We will write a function `chat(message, history)` where:
**message** is the prompt to use
**history** is a list of pairs of user message with assistant's reply

```
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
```
We will convert this history into the prompt style for OpenAI, then call OpenAI. 

In [12]:
def chat(message, history):
    # message: The latest message input from the user.
    # history: A list of tuples containing previous user and assistant messages.

    # Initialize the conversation with a system message that sets the assistant's behavior.
    # The OpenAI API expects messages as a list of dictionaries with "role" and "content" keys.
    messages = [{"role": "system", "content": system_message}]

    # The conversation history is stored as a list of tuples, where each tuple contains a user message and the corresponding assistant reply.
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assitant_message})
    # Add the latest user message to the conversation.
    messages.append({"role": "user", "content": message})

    print("History is: ")
    print(history)
    print("And messages is: ")
    print(messages)

    stream = openai.chat.completions.create(model = MODEL, messages = messages, stream = True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response
    

In [15]:
# gr.ChatInterface: A class provided by the Gradio library specifically designed for creating chat-based interfaces.
# gr.ChatInterface(fn=chat): Creates a chat interface bound to your chat function, which processes messages.
# .launch(): Starts the Gradio app, making the chat interface accessible to users.
gr.ChatInterface(fn = chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.


History is: 
[]
And messages is: 
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hi, please introduce yourself to me. '}]


In [ ]:
({"role": "", "": })